In [7]:
%pylab inline
import pandas as pd
import numpy as np
import time
from sklearn.grid_search import GridSearchCV
from sklearn.ensemble import (RandomForestClassifier, ExtraTreesClassifier, AdaBoostClassifier,\
                              GradientBoostingClassifier)
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn import metrics
from sklearn.linear_model import LogisticRegression

Populating the interactive namespace from numpy and matplotlib


In [2]:
def read_csv(day_trade):
    data_up = []
    data_down = []
    path = '/home/rory/SGX-OrderBook-Tick-Data-Trading-Strategy/Train_Test_Data/ML_data_2014'
    for j,i in enumerate(day_trade):
        for k in range(0,len(i),1):
            path_up = path + '_' + str(j+1) + '_' + str(i[k]) + '_' + 'UP' + '.csv'
            path_down = path + '_' + str(j+1) + '_' + str(i[k]) + '_' + 'DOWN' + '.csv'
            data_up.append(pd.read_csv(path_up))
            data_down.append(pd.read_csv(path_down))
            #print path_down
    return data_up,data_down

In [3]:
SEED = 42
def latest_day_loop_acc(X_train,y_train,X_test,y_test,model):
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    acc = metrics.accuracy_score(y_test, preds)
    return acc

In [4]:
day_trade = [[2,3,6,7,8,10,13,14,15,16]]#,17,20,21,22,23,24,27,28,29,30],\
             #[7,10,11,12,13,17,18,19,21,24,25,26,27],\
             #[3,4,5,6,7,10,11,13,14,17,18,19,20,24,25,26,27,31]]

In [5]:
data_2014_up, data_2014_down = read_csv(day_trade)

In [84]:
models = {
    'RandomForestClassifier': RandomForestClassifier(random_state = 0)#,
    #'ExtraTreesClassifier': ExtraTreesClassifier(random_state = 0),
    #'AdaBoostClassifier': AdaBoostClassifier(base_estimator = DecisionTreeClassifier,n_estimators = 10),
    #'GradientBoostingClassifier': GradientBoostingClassifier(random_state = 0),
    #'SVC': SVC(probability=True,random_state = 0),
    #'LogisticRegression':LogisticRegression()
}

In [101]:
model_grid_params = {
    'RandomForestClassifier': {'max_features':[None],'n_estimators':[10],'max_depth':[10],\
                               'min_samples_split':[2,30],'criterion':['entropy'],\
                               'min_samples_leaf':[5,10]}#,
    #'ExtraTreesClassifier': {'max_features':[None],'n_estimators':[10],'max_depth':[10],\
    #                         'min_samples_split':[2,30],'criterion':['entropy'],\
    #                         'min_samples_leaf':[5,10]},
    #'AdaBoostClassifier': {"base_estimator__criterion" : ["entropy"],\
    #                       "base_estimator__max_depth": [None],\
    #                       "base_estimator__min_samples_leaf" : [5],\
    #                       "base_estimator__min_samples_split" : [2],\
    #                       "base_estimator__max_features" : [None]},
    #'GradientBoostingClassifier': {'max_features':[None],'n_estimators':[10],'max_depth':[10],\
    #                               'min_samples_split':[2,30],'min_samples_leaf':[5,10],\
    #                               'learning_rate':[0.1],'subsample':[1.0]},
    #'SVC': [{'kernel':['rbf'],'gamma':[1e-1,1e-2],'C':[1,10]},\
    #        {'kernel':['linear'],'C':[1,10]}]
}

In [97]:
#params['RandomForestClassifier']

In [164]:
class Model_Selection:
    def __init__(self,models,model_grid_params):
        self.models = models
        self.model_grid = model_grid_params
        self.keys = models.keys()
        self.Best_Score = {}
        
    def model_fit(self,X_train,y_test,cv = 5,scoring = 'accuracy'):#,y_train,X_test
        
        for key in self.keys:
            print "Running GridSearchCV for %s." %(key)
            model = self.models[key]
            model_grid = self.model_grid[key]
            Grid = GridSearchCV(model,model_grid,cv=cv,scoring=scoring)
            %time Grid.fit(X_train,y_train) 
            print Grid#.best_params_
            #self.Best_Score.
            print 'CV Best Score = %s'%(Grid.best_score_)
        
        
        
        
    def print_(self):
        print self.models.keys()

In [165]:
a = Model_Selection(models,model_grid_params)

In [166]:
a.model_fit(X_train,y_train)

Running GridSearchCV for RandomForestClassifier.
CPU times: user 3.44 s, sys: 0 ns, total: 3.44 s
Wall time: 3.57 s
GridSearchCV(cv=5, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=0, verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'min_samples_leaf': [5, 10], 'n_estimators': [10], 'min_samples_split': [2, 30], 'criterion': ['entropy'], 'max_features': [None], 'max_depth': [10]},
       pre_dispatch='2*n_jobs', refit=True, scoring='accuracy', verbose=0)
CV Best Score = 0.757777777778


In [56]:
latest_sec = 60 * 30
pred_sec = 10
day = 0
for i in range(0,10,pred_sec):#9000-latest_sec-pred_sec,pred_sec):
    data_train = data_2014_up[day][i:i+latest_sec]
    X_train = data_train.drop(["0"],axis=1)
    y_train = data_train['0']

In [156]:
e = {}

In [155]:
e['a'].append(2)
e

{'a': [2, 2, 2]}